In [26]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.2.0
import folium
print('librairies imported')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.2.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    folium-0.2.0               |           py35_0          78 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    folium:          0.2.0-py35_0      conda-forge
    vincent:         0.4.4-py_1        conda-forge

The following packages will be UPDATE

In [35]:
!wget -q -O 'toronto_data.json' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
        
print('data downloaded!')

data downloaded!


In [57]:
columns_name= ['PostalCode', 'Borough', 'Neighborhood']
neighborhood_data= pd.DataFrame(columns= columns_name)

In [59]:
neighborhood_data

,PostalCode,Borough,Neighborhood


In [62]:
for data in neighborhood_data:
    postalcode= neighborhood_data ['properties']['postalcode']
    neighborhood_name= data['properties']['name']
    
    neighborhood_name= data[properties][neighborhood]
    neighborhood= neighborhood.append({'PostalCode': postalcode,
                                       'Borough': neighborhood_name,
                                       'neighborhood': neighborhood_name}, ignore_index= True)

KeyError: 'properties'